In [ ]:
import numpy as np # linear algebra
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import math
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader

from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.metrics import accuracy_score

timm_path = "../input/timm-pytorch-image-models/pytorch-image-models-master"
import sys
sys.path.append(timm_path)
import timm

import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
class GeM2(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM2,self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

In [ ]:
df  = pd.read_csv("../input/rarity-folds/rarity_folds.csv")
df.head()

In [ ]:
train_aug = A.Compose(
    [  

        A.Resize(490,490,p=1.0),
        A.RandomCrop(448,448),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Transpose(p=0.5),
        A.Rotate(limit=120, p=0.5),
            A.RandomBrightness(limit=(0.09, 0.6), p=0.5),
        A.Normalize(p=1.0),
        ToTensorV2(p=1.0)
    ]
)
val_aug = A.Compose(
    [  

        A.Resize(width=512, height=512, p=1.0),
     A.Normalize(p=1.0),
        ToTensorV2(p=1.0)
    ]
)

In [ ]:
class Shop(Dataset):
    def __init__(self,df,augs=None):
        self.df = df
        self.augs = augs
        
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        img_src = self.df.path.iloc[idx]
        image = cv2.imread(img_src)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.uint8)
        
        p_id =  self.df.posting_id.iloc[idx]
        
        if (self.augs):
            transformed = self.augs(image=image)
            image = transformed['image']
        
        label = self.df.iloc[idx].rarity	
        label =torch.tensor(label, dtype=torch.long)
        
        return image,label,p_id

In [ ]:
class Model(nn.Module):
    def __init__(self,output_size =10 ,pretrained=False):
        super().__init__()
        self.op = output_size
        self.backbone = timm.create_model('tf_efficientnet_b2_ns', features_only=True, 
                                          pretrained=pretrained)
        self.gem2 = GeM2()
        self.fc1 = nn.Linear(352,144)
        self.fc2 = nn.Linear(144,self.op)
        self.do = nn.Dropout(p=0.25)
        self.bn1 = nn.BatchNorm1d(352)
        self.bn2 = nn.BatchNorm1d(144)
        self.pl = nn.PReLU()
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)

    def forward(self,x,labels=None):
        y = self.backbone(x)
        #y1 = self.gem1(y[2])
        y2 = self.gem2(y[4])
        
        #y1 = y1.view(x.shape[0],-1)
        y2 = y2.view(x.shape[0],-1)
        y2 = self.bn1(y2)
        
        #concat = torch.cat((y1,y2),dim = 1)
        fc1 = self.do(self.fc1(y2))
        feat1 = self.pl(self.bn2(fc1))
        
        feat1 = self.fc2(feat1)
        
        return feat1
        
       


In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def train_one_epoch(train_loader,model,optimizer,criterion,e,epochs,scheduler):
    losses = AverageMeter()
    scores = AverageMeter()
    
    model.train()
    global_step = 0
    loop = tqdm(enumerate(train_loader),total = len(train_loader))
    
    for step,(image,labels,_) in loop:
        image = image.to(device)
        labels= labels.to(device)
        logitss = model(image)
        batch_size = labels.size(0)
        loss  = criterion(logitss,labels)
        
        out = logitss.softmax(1)
        outputs = torch.argmax(out, dim=1).cpu().detach().numpy()
        targets = labels.cpu().detach().numpy()
        accuracy = accuracy_score(targets, outputs)
        
        losses.update(loss.item(), batch_size)
        scores.update(accuracy.item(), batch_size)
        
        optimizer.zero_grad()
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(m.parameters(), 1000 )
        optimizer.step()
        scheduler.step() 
        global_step += 1
        
        loop.set_description(f"Epoch {e+1}/{epochs}")
        loop.set_postfix(loss = loss.item(), accuracy = accuracy.item(), stage = 'train')
        
        
    return losses.avg,scores.avg

In [ ]:
def val_one_epoch(loader,model,optimizer,criterion):
    losses = AverageMeter()
    scores = AverageMeter()
    model.eval()
    global_step = 0
    loop = tqdm(enumerate(loader),total = len(loader))
    
    for step,(image,labels,_) in loop:
        image = image.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            output = model(image)
        loss = criterion(output,labels)
        
        output = output.softmax(1)
        outputs = torch.argmax(output, dim=1).cpu().detach().numpy()
        targets = labels.cpu().detach().numpy()
        accuracy = accuracy_score(targets, outputs)
        
        losses.update(loss.item(), batch_size)
        scores.update(accuracy.item(), batch_size)
        loop.set_postfix(loss = loss.item(), accuracy = accuracy.item(), stage = 'valid')
        
        global_step += 1
        
    
        
    return losses.avg,scores.avg

In [ ]:
model = Model(pretrained=True)
model.to(device);

In [ ]:
import os
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
def fit(fold):
  
    df_train = df[(df.fold == 1) | (df.fold == 2) | (df.fold == 4) ].reset_index(drop=True)
    df_valid = df[(df.fold == 0) | (df.fold == 3)].reset_index(drop=True)
    
    train_data = Shop(df_train,augs= train_aug)
    val_data   = Shop(df_valid,augs=val_aug)
    
    train_loader = DataLoader(train_data,shuffle=True,
                        num_workers=4,
                        batch_size=16,
                        drop_last=True,
                            pin_memory=True)
    
    val_loader = DataLoader(val_data,shuffle=False,
                        num_workers=4,
                            pin_memory=True,
                        batch_size=16)
    criterion= nn.CrossEntropyLoss()
    epochs = 8
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4 , weight_decay = 1e-4)
    
    num_train_steps = math.ceil(len(train_loader))
    warmup_epochs = 1
    num_warmup_steps= num_train_steps * warmup_epochs
    num_training_steps=int(num_train_steps * epochs)
    scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps = num_warmup_steps,num_training_steps =num_training_steps)
    
    best_acc = 0
    loop = range(epochs)
    for e in loop:
        
        train_loss,train_accuracy = train_one_epoch(train_loader,model,optimizer,criterion,e,epochs,scheduler)
        print(f'For epoch {e+1}/{epochs}')
        print(f'average train_loss {train_loss}')
        print(f'average train_accuracy {train_accuracy}' )
        
        val_loss,val_accuracy = val_one_epoch(val_loader,model,optimizer,criterion)
        
        scheduler.step(val_loss)
        
        print(f'avarage val_loss { val_loss }')
        print(f'avarage val_accuracy {val_accuracy}')
        
        if (val_accuracy>best_acc):
            best_acc = val_accuracy
            print(f'saving model for {best_acc}')
            torch.save(model.state_dict(),OUTPUT_DIR+ f'Fold {fold} model with val_acc {best_acc}.pth') 
        
        
    
    
fit(0)

In [ ]:
torch.save(model.state_dict(),'image_modelT1_256_seresnext.pth') 
